In [114]:

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
# imprt tree
from sklearn import tree
# import metrics
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder

In [95]:
df_raw = pd.read_csv("./labeled_data/labeled_df.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",", index_col=False,
                    usecols=['SessionID','Activity','anomaly'])
# df_raw["TIMESTAMP"] = pd.to_datetime(df_raw["TIMESTAMP"], infer_datetime_format=True)
# df_raw = df_raw.sort_values(by=["SessionID", "TIMESTAMP"]).copy()
df_raw

,SessionID,Activity,anomaly
0,46,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
1,46,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,False
2,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,False
3,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
4,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
...,...,...,...
7174929,55314751,/werk_nl/werknemer/werkmap,False
7174930,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
7174931,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
7174932,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False


In [96]:
# change anomaly == True to 1 and False to 0
df_raw["anomaly"] = df_raw["anomaly"].astype(int)

In [97]:
# count the number of unique SessionID where anomaly == True
count_anomaly_raw = df_raw[df_raw["anomaly"] == 1]["SessionID"].nunique()
count_normal_raw = df_raw[df_raw["anomaly"] == 0]["SessionID"].nunique()
print(f'Amount of anomalous sessions in the dataset:   {count_anomaly_raw}' )
print(f'Amount of normal sessions in the dataset:      {count_normal_raw}')
distribution =  count_anomaly_raw / count_normal_raw 
print(f'Distribution:                                  {distribution * 100} %')

Amount of anomalous sessions in the dataset:   355
Amount of normal sessions in the dataset:      659915
Distribution:                                  0.05379480690695014 %


In [98]:
df_anomaly = df_raw[df_raw["anomaly"] == 1].copy()
df_normal = df_raw[df_raw["anomaly"] == 0].copy()
nr_of_sessions_used = 50000
injection_rate = nr_of_sessions_used / count_normal_raw
injection_amount = int(injection_rate * count_anomaly_raw)
print(injection_rate)
print(injection_amount)
# get 20 random sessionIDs from the anomaly dataset
anomaly_sessionIDs = df_anomaly["SessionID"].sample(n=injection_amount, random_state=1).tolist()




0.07576733367176075
26


In [99]:
# pick random int > 0 and < (len(df_raw) - nr_of_sessions_used)
lower = np.random.randint(0, len(df_raw) - nr_of_sessions_used)
upper = lower + nr_of_sessions_used
print(f'Lower bound: {lower}')
print(f'Upper bound: {lower + nr_of_sessions_used}')
# get a random sample of the sessions
# df_ = df_raw[df_raw["SessionID"].isin(df_raw["SessionID"].unique()[:nr_of_sessions_used])].copy()
df_ = df_normal[df_normal["SessionID"].isin(df_normal["SessionID"].unique()[:nr_of_sessions_used])].copy()
df_ = df_.append(df_anomaly[df_anomaly["SessionID"].isin(anomaly_sessionIDs)], ignore_index=True)
# Only use the columns that are needed -> SessionID, Activity, anomaly
df_50k = df_[["SessionID", "Activity", "anomaly"]]


# remove the sessions with ID in anomaly_sessionIDs from the df_anomaly dataset
df_anomaly = df_anomaly[~df_anomaly["SessionID"].isin(anomaly_sessionIDs)].copy()
df_anomaly.to_csv("gen_sessions/an.csv", index=False)

Lower bound: 4466611
Upper bound: 4516611


In [100]:
# count the number of unique SessionID where anomaly == True
count_anomaly = df_50k[df_50k["anomaly"] == 1]["SessionID"].nunique()
count_normal = df_50k[df_50k["anomaly"] == 0]["SessionID"].nunique()
print(f'Amount of anomalous sessions in the sampled dataset:   {count_anomaly}')
print(f'Amount of normal sessions in the sampled dataset:      {count_normal}')
distribution =  count_anomaly / count_normal 
print(f'Distribution:                                          {distribution * 100} %')

Amount of anomalous sessions in the sampled dataset:   26
Amount of normal sessions in the sampled dataset:      50000
Distribution:                                          0.052 %


Helper Functions

In [101]:
# function to encode the activities
def encode_activities(_df):
    df = _df.copy()
    le = LabelEncoder()
    le.fit(df)
    df= le.transform(df)
    return df

# function to create a list of al the activities
def activities_list(_df):
  df = _df.copy()
  activities = []
  for i in df["Activity"].unique():
    activities.append(i)
  return activities

# function to create a dictionary with the activities as keys and the one hot encoded activities as values
def create_dict(l1, l2):
  dic = {}
  for i in range(len(l1)):
    dic[l1[i]] = l2[i]
  return dic

# function to transform each session into a sequence of activities
def transform_session_to_sequence(_df):
  df = _df.copy()
  df_ses = df.groupby("SessionID")["Activity"].apply(list).reset_index()
  # merge the anomaly column
  df_ses = df_ses.merge(df[["SessionID", "anomaly"]].drop_duplicates(), on="SessionID")
  return df_ses

# function the create the dataFrame for the model
def df_for_model(_df,Activity_col):
  df = pd.DataFrame(_df[Activity_col].values.tolist(), index= _df.index)
  df_for_model = pd.concat([_df, df], axis=1)
  df_for_model = df_for_model.drop(columns=[Activity_col])
  df_for_model.fillna(0, inplace=True)
  return df_for_model
  
# function to concat 2 dataframes
def concat_df(df_gen ,amount_real,amount_generated,_df1 = df_raw,):
  df1 = _df1.copy()
  df2 = df_gen
  df_raw_filtered = df1[df1["SessionID"].isin(df1["SessionID"].unique()[:amount_real])]
  df_gen_anomalies_filtered = df2[df2["SessionID"].isin(df2["SessionID"].unique()[:amount_generated])]
  df = pd.concat([df_raw_filtered, df_gen_anomalies_filtered], ignore_index=True)
  return df

GLOBALS

In [102]:
# Global variables
# list of all possible activities in df_raw
ALL_ACTIVITIES = activities_list(df_raw)
ENCODED_ACTIVITIES = encode_activities(ALL_ACTIVITIES)
DICT = create_dict(ALL_ACTIVITIES, ENCODED_ACTIVITIES)
if len(ALL_ACTIVITIES) == len(ENCODED_ACTIVITIES) == len(DICT):
  print("All lists are the same length")


All lists are the same length


Stappenplan:

1. df -> transform_session_to_sequence
2. Transform Activities to encoded Activities
3. Transform the list in the Activity column to multiple columns with fillna(0)

In [103]:
base_data = transform_session_to_sequence(df_50k)
# base_an = transform_session_to_sequence(df_50k)

In [104]:
base_data['Activity'] = base_data['Activity'].apply(lambda x: [DICT[i] for i in x])
# base_an['Activity'] = base_an['Activity'].apply(lambda x: [DICT[i] for i in x])

In [105]:
base_data = df_for_model(base_data, "Activity")
# base_an = df_for_model(base_an, "Activity")

In [106]:
base_data
# drop the SessionID column
base_data = base_data.drop(columns=["SessionID"])
base_data

,anomaly,0,1,2,3,4,5,6,7,8,...,355,356,357,358,359,360,361,362,363,364
0,0,372,371.0,392.0,391.0,391.0,391.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,391,372.0,401.0,391.0,391.0,391.0,391.0,369.0,367.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,372,372.0,372.0,372.0,372.0,372.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,391,391.0,391.0,391.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50021,1,402,402.0,402.0,402.0,402.0,402.0,165.0,367.0,622.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50022,1,367,365.0,398.0,365.0,365.0,398.0,398.0,398.0,398.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50023,1,353,41.0,41.0,367.0,41.0,41.0,41.0,41.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50024,1,367,391.0,392.0,392.0,372.0,372.0,372.0,371.0,371.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
# make a list of the values in column 0 to n of base_data and put in column seq
base_data['seq'] = base_data.values.tolist()
# drop the first value of the list in seq
base_data['seq'] = base_data['seq'].apply(lambda x: x[1:])

0        [372, 371.0, 392.0, 391.0, 391.0, 391.0, 0.0, ...
1        [369, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2        [391, 372.0, 401.0, 391.0, 391.0, 391.0, 391.0...
3        [372, 372.0, 372.0, 372.0, 372.0, 372.0, 0.0, ...
4        [391, 391.0, 391.0, 391.0, 0.0, 0.0, 0.0, 0.0,...
                               ...                        
50021    [402, 402.0, 402.0, 402.0, 402.0, 402.0, 165.0...
50022    [367, 365.0, 398.0, 365.0, 365.0, 398.0, 398.0...
50023    [353, 41.0, 41.0, 367.0, 41.0, 41.0, 41.0, 41....
50024    [367, 391.0, 392.0, 392.0, 372.0, 372.0, 372.0...
50025    [367, 393.0, 393.0, 393.0, 393.0, 393.0, 393.0...
Name: seq, Length: 50026, dtype: object

Functions to run the model

In [107]:
# Function to split the data into train and test data
def split_data(_df):
  df = _df.copy()
  X = df.drop(columns=["anomaly"])
  y = df["anomaly"]
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

In [108]:
# Base test and train data
X_train, X_test, y_train, y_test = split_data(base_data)

Run the model

In [112]:
# import the generated sessions:
base_path = "gen_sessions/"
gen_sessions_paths = [
  # base_path + '5_1000.csv',
  # base_path + '10_1000.csv',
  # base_path + '25_1000.csv',
  # base_path + '50_1000.csv',
  # base_path + '75_1000.csv',
  # base_path + '100_1000.csv',
  base_path + 'an.csv',
]

In [110]:
auc_score_dicts = {}
dict_prec_and_rec_list = {}

amount_anomalies_list_total = []
precision_score_list_total = []
recall_score_list_total = []

In [113]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score


for sessions in gen_sessions_paths:
    amount_anomalies_list = []
    precision_score_list = []
    recall_score_list = []
    
    # build the dataFrame
    cvs = pd.read_csv(sessions,index_col=False)
    # check if column Unnamed: 0 exists
    if "Unnamed: 0" in cvs.columns : cvs = cvs.drop(columns=["Unnamed: 0"])
    cvs['anomaly'] = 1
    cvs.rename(columns={'URL_FILE':'Activity'}, inplace=True)
    gen_df_seq = transform_session_to_sequence(cvs)
    
    gen_df_seq['Activity'] = gen_df_seq['Activity'].apply(lambda x: [DICT[i] for i in x])
    ready_df = df_for_model(gen_df_seq, "Activity")
    ready_df = ready_df.drop(columns=["SessionID"])
    ready_df = ready_df.sample(frac=1).reset_index(drop=True) 
    
    for amount_gen in [10, 20, 50, 65, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]:
    # from 0 to 250 with a step of 25
    # for amount_gen in [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]:
        amount_real = 0
        df_gen = ready_df[:amount_gen]
        # print(df_gen)
        # Add the generated anomalies to the training dataset
        X_train_extra = pd.concat([X_train, df_gen.drop(columns=["anomaly"])]).fillna(0)
        y_train_extra = pd.concat([y_train, df_gen["anomaly"]])
        
        clf = tree.DecisionTreeClassifier().fit(X_train_extra, y_train_extra)
        # clf_extra = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_extra, y_train_extra_transformed)
        #y_train_predict = clf_extra.predict(X_train_extra)
        predictions = clf.predict(X_test)
        
        print(sessions)
        print(amount_gen)
        print(accuracy_score(y_test, predictions))
        print(balanced_accuracy_score(y_test, predictions))
        print()
    
       
        


gen_sessions/an.csv
10
0.9985008994603238
0.4995000499950005

gen_sessions/an.csv
20
0.9976014391365181
0.599000099990001

gen_sessions/an.csv
50
0.9977013791724965
0.599050094990501

gen_sessions/an.csv
65
0.997101738956626
0.4988001199880012

gen_sessions/an.csv
100
0.9972016789926045
0.4988501149885011

gen_sessions/an.csv
200
0.994203477913252
0.5973002699730027

gen_sessions/an.csv
300
0.9938037177693384
0.597100289971003

gen_sessions/an.csv
400
0.9916050369778133
0.596000399960004

gen_sessions/an.csv
500
0.9917049770137917
0.5960503949605039

gen_sessions/an.csv
600
0.9906056366180291
0.6954504549545046

gen_sessions/an.csv
700
0.9909054567259644
0.5956504349565044

gen_sessions/an.csv
800
0.9915050969418349
0.595950404959504

gen_sessions/an.csv
900
0.9912052768338997
0.6957504249575043

gen_sessions/an.csv
1000
0.991305216869878
0.5958504149585041

